In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = pd.read_csv('C:\\Users\\ankur_vc1xnom\\Desktop\\LP4\\LP4\\ecg.csv')

In [ ]:
df.head()

In [ ]:
column_names = [i for i in range(141)] 
df.columns = column_names

In [ ]:
df.columns

In [ ]:
# Splitting the dataset into features and target
features = df.drop(140, axis=1)  # Features are all columns except the last (column 140)
target = df[140]  # Target is the last column (column 140)

# Split the data into training and testing sets (80% training, 20% testing)
x_train, x_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2
)

# Get the indices of the training data points labeled as "1" (anomalies)
train_index = y_train[y_train == 1].index

# Select the training data points that are anomalies
train_data = x_train.loc[train_index]

In [ ]:
# Initialize the Min-Max Scaler to scale the data between 0 and 1
min_max_scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the training data
x_train_scaled = min_max_scaler.fit_transform(train_data.copy())

# Scale the testing data using the same scaler
x_test_scaled = min_max_scaler.transform(x_test.copy())

In [ ]:
# Creating an Autoencoder model by extending the Model class from Keras
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout
class AutoEncoder(Model):
    def __init__(self, output_units, ldim=8):
        super().__init__()
        # Define the encoder part of the Autoencoder
        self.encoder = Sequential([
            Dense(64, activation='relu'),
            Dropout(0.1),
            Dense(32, activation='relu'),
            Dropout(0.1),
            Dense(16, activation='relu'),
            Dropout(0.1),
            Dense(ldim, activation='relu')
        ])
        # Define the decoder part of the Autoencoder
        self.decoder = Sequential([
            Dense(16, activation='relu'),
            Dropout(0.1),
            Dense(32, activation='relu'),
            Dropout(0.1),
            Dense(64, activation='relu'),
            Dropout(0.1),
            Dense(output_units, activation='sigmoid')
        ])

    def call(self, inputs):
        # Forward pass through the Autoencoder
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
model = AutoEncoder(output_units=x_train_scaled.shape[1])

model.compile(loss='msle', metrics=['mse'], optimizer='adam')

history = model.fit(
    x_train_scaled,  
    x_train_scaled,  
    epochs=20,        
    batch_size=512,   
    validation_data=(x_test_scaled, x_test_scaled),  
    shuffle=True     
)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()

In [ ]:
def find_threshold(model, x_train_scaled):
    recons = model.predict(x_train_scaled)

    recons_error = tf.keras.metrics.msle(recons, x_train_scaled)

    threshold = np.mean(recons_error.numpy()) + np.std(recons_error.numpy())

    return threshold

def get_predictions(model, x_test_scaled, threshold):
    predictions = model.predict(x_test_scaled)

    errors = tf.keras.losses.msle(predictions, x_test_scaled)

    anomaly_mask = pd.Series(errors) > threshold

    preds = anomaly_mask.map(lambda x: 0.0 if x == True else 1.0)

    return preds

threshold = find_threshold(model, x_train_scaled)
print(f"Threshold: {threshold}")

In [ ]:
from sklearn import metrics
predictions = get_predictions(model, x_test_scaled, threshold)

accuracy = metrics.accuracy_score(predictions, y_test)

print(f"Accuracy Score: {accuracy}")